In [1]:
import time
import datetime
from math import exp, log,sqrt
import numpy as np
import pandas as pd
from random import gauss
from matplotlib import pyplot as plt


In [2]:
# We create the date range between two date by taking out the holidays and weekends.
from pandas.tseries.holiday import USFederalHolidayCalendar
from datetime import timedelta
cal = USFederalHolidayCalendar()

def daterange(date1, date2):
    holidays = pd.to_datetime(cal.holidays(start=date1, end=date2))
    date_range = [date1]
    for n in range(int ((date2 - date1).days)+1):
        date_next =  date1 + timedelta(n)
        if (date_next not in holidays) and (date_next.weekday() < 5):
            date_range.append(date_next)
    return date_range

In [3]:
def generate_paths(S_t1,df_t1,v,div,r_t1,kappa_r,theta_r,sigma_r,t1,t2):
    #S_t1: price at the date t1
    #df_t1: discount factor at the date t1
    #r_t1: interest rate at the date t1
    #kappa_r, theta_r, sigma_r: the parameters of model stochastic interest rate
    #t1: the date t1
    #t2: the date t2
    
    dates = daterange(t1, t2)
    M = len(dates)
    dt= 1/252
    rand_r = np.random.standard_normal(M)
    rand_S = np.random.standard_normal(M)
    S = np.zeros_like(rand_S)
    r = np.zeros_like(rand_r)
    S[0] = S_t1
    r[0] = r_t1
    for i in range(1,M):
        r[i] = max(r[i-1] + kappa_r * (theta_r - r[i- 1]) *dt + sigma_r *sqrt(r[i- 1]) * sqrt(dt) * rand_r[i], 0)
        S[i] = S[i - 1] * exp((r[i-1]-div - v ** 2 / 2) * dt + v  * sqrt(dt) * rand_S[i])
    S_t2 = S[-1] # the asset price at the date t2
    df_t2 = df_t1 * exp(-(sum(r) - r[-1]) * dt ) #discount factor for the date t2
    r_t2  = r[-1] # short term interest rate at the the date t2

    return [S_t2, df_t2, r_t2]
    #S_t2: asset price at the date t2
    #df_t2: discount factor at the date t2
    #r_t2: interest rate at the date t2
    
    
    

In [4]:
# We import the anniversary dates
tabDate = pd.read_csv('date.csv', sep= ";", infer_datetime_format= True)
date= [datetime.datetime.strptime(x, '%d/%m/%Y') for x in list(tabDate.date)]

In [5]:
S0 =100# We set asset price at the start date
v= 0.2 # Implied volatility
div = 0.00 # Dividend rate
kappa_r =0.3 #parameter
theta_r = 0.04# long term interest rate
sigma_r = 0.1 # vol of interest rate
r0 = 0.04 # short term rate at the start date
N = 1000 # Notional
coupon_barrier = 0.8
kickout_barrier = 1.1
protection_barrier = 0.6
coupon_rate = 0.088

autocall_prices= []# We stock all of autocall prices (at the start_date ) generated by all the pathes
l = len(date)
S = [S0] * l # Asset prices
df = [1] * l # Discount factors
r= [r0] * l # Interest rate

In [6]:
def mcSimulation(simulations):
    start_date = date[0]
    end_date = date[-1]
    for i in range(simulations):
        autocall_price = 0
        checker = True
        payoffs = [0] * l # we set all the payoff = 0
        #The payoff at the start_date is always zero

        for j in range(1, l- 1) :
            X= generate_paths(S[j-1], df[j-1],v,div,r[j-1],kappa_r,theta_r,sigma_r,date[j-1],date[j])
            S[j] = X[0] # The asset price at jTH anniversary
            df[j]= X[1] # Discount factor at jTH anniversary
            r[j] = X[2] # Short term interest rate at jTH anniversary
            if S[j]/S0 < coupon_barrier:
                payoffs[j+1] = payoffs[j]+ N * coupon_rate
                payoffs[j] = 0
                autocall_price += payoffs[j] * df[j]
            elif S[j]/S0 < kickout_barrier:
                payoffs[j] += N * coupon_rate
                autocall_price += payoffs[j]  * df[j]
            else:
                payoffs[j] += N * coupon_rate + N
                autocall_price += payoffs[j] * df[j]
                checker = False
                break

        if checker: 
            X= generate_paths(S[-2], df[-2],v,div,r[-2],kappa_r,theta_r,sigma_r,date[-2],date[-1])
            S[-1] = X[0] # The asset price at jTH anniversary
            df[-1]= X[1] # Discount factor at jTH anniversary
            r[-1] = X[2] # Short term interest rate at jTH anniversary
            if S[-1]/S0 < protection_barrier:
                payoffs[-1] = N * S[-1]/S0
            elif S[-1]/S0 < coupon_barrier: 
                payoffs[-1] = N
            else:
                payoffs[-1] += N * coupon_rate + N
            autocall_price += payoffs[-1] * df[-1]

        autocall_prices.append(autocall_price)

    final_autocall_price = sum(autocall_prices)/float(simulations)
    return final_autocall_price

In [7]:
autocall = mcSimulation(1000)
print(autocall)

1050.0871702731731
